In [ ]:
import numpy as np 
import pandas as pd 
import math, os
from sklearn import preprocessing
import gmplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, Masking, Input, Dropout
from keras.layers.merge import concatenate
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint
pd.set_option('display.max_columns', None)

In [ ]:
def obtain_data():
    dfs={}
    for i in range(19):
        year=str(2000+i)
        file='datos/DesmatamentoMunicipios' + year + '.csv'
        dfs[i] = pd.read_csv(file, encoding = "ISO-8859-1", index_col=0, sep=",")
        #eliminamos columnas irrelevantes
        dfs[i].drop(columns='Latgms', inplace=True)
        dfs[i].drop(columns='Longms', inplace=True)
        dfs[i].drop(columns='CodIbge', inplace=True)
        #df.rename(columns={0:'Latitud', 1:'Longitud', 2:'Municipio', 3:'Estado', 4: 'AreaKm2', 5:'Deforestacion', 6:'Incremento deforestacion', 7:'Bosque', 8:'Nubes', 9:'No observado', 10:'No bosque', 11:'Hidrografia', 12:'Check'} ,inplace=True)
        dfs[i].columns=['Latitud', 'Longitud', 'Municipio', 'Estado', 'Area total', 'Deforestacion ' + year, 'Incremento deforestacion ' + year, 'Bosque ' + year, 'Nubes ' + year, 'No observado ' + year, 'No bosque', 'Hidrografia', 'Check ' + year]
        dfs[i]=dfs[i][['Latitud', 'Longitud', 'Municipio', 'Estado', 'Area total', 'No bosque', 'Hidrografia', 'Deforestacion ' + year, 'Incremento deforestacion ' + year, 'Bosque ' + year, 'Nubes ' + year, 'No observado ' + year, 'Check ' + year]]
    return dfs

In [ ]:
def create_df(dfs):
    df=dfs[0]
    dic={}
    for idx, row in df.iterrows():
            if row['Municipio'] in dic:
                dic[row['Municipio']]+=1
            else:
                dic[row['Municipio']]=1

    repetidos=[]
    for key, value in dic.items():
        if value==2:
            repetidos.append(key)

    rep= df.Municipio.isin(repetidos)
    mismo_municipio =  df[rep]
    print("Hay algunos municipios con el mismo nombre en 2 estados distintos:")
    print(mismo_municipio.loc[:,'Latitud':'Area total'])

    df=dfs[0]
    for idx, value in dfs.items():
        value['Municipio']=value['Municipio'] + " (" + value['Estado'] + ")"
        if idx > 0:
            value.drop(columns='Latitud', inplace=True)
            value.drop(columns='Longitud', inplace=True)
            value.drop(columns='Estado', inplace=True)
            value.drop(columns='Area total', inplace=True)
            value.drop(columns='Hidrografia', inplace=True)
            value.drop(columns='No bosque', inplace=True)
            df=pd.merge(df, value, on='Municipio')

    # reorder columns
    cols = df.columns.tolist()
    cols = cols[2:4] + cols[0:2] + cols[4:]
    df = df[cols]

    print("Por eso, hacemos merge por la dupla municipio-estado:")
    print(df.head())
    return df

In [ ]:
def shuffle_df(df):
    print("El dataset viene ordenado por deforestacion, de mayor a menor. Para entrenar el modelo alteramos el orden de forma aleatoria.")
    df = df.sample(frac=1)
    df.reset_index(inplace=True, drop=True)
    df.head()
    return df

In [ ]:
def plot_map(df, name):
    print("Dibujamos el mapa de " + str(len(df['Latitud'])) + " municipios")
    gmap = gmplot.GoogleMapPlotter(df['Latitud'].values[0], df['Longitud'].values[0], 5) # coordenadas del primer municipio del dataframe
    gmap.heatmap(df['Latitud'], df['Longitud']) 
    gmap.draw('mapas/' + name)
    print("Mapa de calor generado")

In [ ]:
def generate_fake_data(df, n_replic):
    if n_replic < 1:
        return
    fake_data=[]
    std=df.std()
    #print(std)
    for idx, row in df.iterrows():
        for i in range(n_replic):
            munic = row['Municipio']+' fake_'+str(i)
            estado = row['Estado']
            lat = row['Latitud']+std['Latitud']*0.1*np.random.uniform(-1,1) # variacion del 10% de la desviación típica
            long = row['Longitud']+std['Longitud']*0.1*np.random.uniform(-1,1) # variacion del 10% de la desviación típica
            area = row['Area total']+row['Area total']*0.5*np.random.uniform(-1,1) # variacion del 50% del valor real
            no_bosq = row['No bosque']+row['No bosque']*0.5*np.random.uniform(-1,1) # variacion del 10% del valor real
            hidro = row['Hidrografia']+row['Hidrografia']*0.5*np.random.uniform(-1,1) # variacion del 10% del valor real
            temp = []
            for j in range(19):    #TODO: Normalizar por año y append todas a la vez, no se puede cambiar el area cada año
                #year = 2000 + j
                if len(temp) > 5:
                    incr = row[n_vars_temp*j+8]   # 7 vars fijas + 1 defor para llegar al incremento
                    proporcion = incr/row['Area total']
                    t = area*proporcion
                    incr = t+t*0.05*np.random.uniform(-1,1)
                    defor = temp[-6] + incr
                    bosque = temp[-4] - incr
                    temp.extend([defor,incr, bosque])
                else:
                    proporcion = row[j+7]/row['Area total']
                    t = area*proporcion
                    defor = t+t*0.05*np.random.uniform(0,1)
                    bosque = area - (defor + no_bosq + hidro)
                    temp.extend([defor,float('NaN'), bosque])
                nubes = 0
                no_obs = 0
                suma = defor + no_bosq + hidro + bosque + nubes + no_obs
                check = suma/area*100
                temp.extend([nubes, no_obs, check])
            fixed = [munic, estado, lat, long, area, no_bosq, hidro]     
            fake_row = np.concatenate((fixed, temp))
            fake_data.append(fake_row)
    fake_df = pd.DataFrame(fake_data, columns=df.columns)
    fake_df = fake_df.infer_objects()
    return fake_df



In [ ]:
def series_to_supervised(temporal_vars, year_ini_train=0, year_ini_test=1, window_size=18, n_vars_temp=6):
    # ensure all data is float
    temporal_vars = temporal_vars.astype('float32')
    temporal_train = temporal_vars.iloc[:, n_vars_temp*year_ini_train:n_vars_temp*(year_ini_train+window_size)]
    temporal_test = temporal_vars.iloc[0:760, n_vars_temp*year_ini_test:] #testeamos con municipios reales
    return temporal_train, temporal_test

In [ ]:
def preprocess_train(temporal_train, fixed_vars, n_vars_temp):
    # data structure for LSTM
    # normalize features
    x = temporal_train.iloc[:,0:-n_vars_temp].values
    y = temporal_train.iloc[:,-n_vars_temp].values.reshape(-1, 1) # reshape in 2D, we get 'deforestacion' as y
    scaler = MinMaxScaler(feature_range=(0, 1))
    x = scaler.fit_transform(x)
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    y = scaler2.fit_transform(y)
    #print(temporal_train.shape, x.shape, y.shape)
    # structure in arrays to be the input of the LSTM
    vars_lstm=[]
    municipio_len=len(x[0])
    for mun in x:
        municipio=[]
        j=0
        while j < municipio_len:
            m=mun[j:j+n_vars_temp]
            if j==0:
                m[1]=-1		# marking missing values: Incremento deforestacion 1999/2000
            #elif m[5]!=municipio[0][5] or m[6]!=municipio[0][6]: # Hidrografia y No bosque son variables fijas!!!
            #	print("Hidrografia y No bosque varian de año en año")
            municipio.append(mun[j:j+n_vars_temp])
            j+=n_vars_temp
        vars_lstm.append(municipio)

    vars_lstm=np.array(vars_lstm)
    X1 = vars_lstm

    # data structure for Dense
    # One hot encoding 
    fixed_vars = pd.concat([fixed_vars,pd.get_dummies(fixed_vars['Estado'])],axis=1)
    # Drop column as it is now encoded
    fixed_vars = fixed_vars.drop('Estado',axis = 1)
    scaler3 = MinMaxScaler(feature_range=(0, 1))
    x = scaler3.fit_transform(fixed_vars.iloc[:,1:]) # nombre del municipio en la primera columna
    X2 = np.array(x)
    return X1, X2, y, scaler, scaler2, scaler3

In [ ]:
def create_model(window_size, n_vars_temp, hidden_layers, mask=1):
    # LSTM para variables temporales
    input_temporal = Input(shape=(window_size-1, n_vars_temp))
    if mask==0:
        lstm = LSTM(5, activation='relu')(input_temporal) #, unroll =True) -> check what is is
    else:
        masking = Masking(mask_value=-1)(input_temporal)
        lstm = LSTM(5, activation='relu', dropout=0.2)(masking)
        #lstm_1 = LSTM(5, activation='relu', return_sequences = True)(masking)
        #lstm_2 = LSTM(5, activation='relu')(lstm_1)
    dense_1 = Dense(16, activation = 'relu')(lstm)
    
    # Dense para variables fijas
    input_fijo = Input(shape=(X2.shape[1],))
    dense_2 = Dense(32, activation = 'relu')(input_fijo)
    
    # concateno las 2 redes
    merge = concatenate([dense_1, dense_2])
    dense = [merge]
    for i in range(hidden_layers):
        dense.append(Dense(16, activation = 'relu')(dense[-1]))
    output = Dense(1, activation = 'relu')(dense[-1])
    model = Model(inputs=[input_temporal, input_fijo], outputs=output)
    
    # summarize layers
    print(model.summary())
    # plot graph
    plot_model(model, to_file='modelos/model_'+str(hidden_layers)+'.png')
    return model

In [ ]:
def preprocess_test(temporal_test, n_vars_temp, scaler, scaler2, scaler3):
    # data structure for LSTM
    # normalize features
    x = temporal_test.values[:,0:-n_vars_temp]
    y = temporal_test.values[:,-n_vars_temp].reshape(-1, 1) # reshape in 2D
    x = scaler.fit_transform(x)
    y = scaler2.fit_transform(y)
    # structure in arrays to be the input of the LSTM
    vars_lstm=[]
    municipio_len=len(x[0])
    for mun in x:
        municipio=[]
        j=0
        while j < municipio_len:
            m=mun[j:j+n_vars_temp]
            if j==0:
                m[1]=-1		# marking missing values: Incremento deforestacion 1999/2000
            #elif m[5]!=municipio[0][5] or m[6]!=municipio[0][6]: # Hidrografia y No bosque son variables fijas!!!
            #	print("Hidrografia y No bosque varian de año en año")
            municipio.append(mun[j:j+n_vars_temp])
            j+=n_vars_temp
        vars_lstm.append(municipio)

    vars_lstm=np.array(vars_lstm)
    X1 = vars_lstm
    return X1, y

In [ ]:
def evaluate_test(municipios_reales, y, y_pred):
    # rescaling
    y=scaler2.inverse_transform(y).flatten()
    y_pred=scaler2.inverse_transform(y_pred).flatten()
    
    errores = []
    abajo = 0
    num_test = len(y_pred)
    for i in range(num_test):
        delta = y[i] - y_pred[i]
        if delta > 0:
            abajo += 1
        error = abs(delta)
        #print(municipios_reales[i], 'Expected', y[i], 'Predicted', y_pred[i], 'Error', str(error))
        errores.append(error)
    test = {'Municipio': municipios_reales.values, 'Expected': y[0:760], 'Predicted': y_pred, 'Error': errores}
    test = pd.DataFrame(test)
    print('Error absoluto medio', np.array(errores).mean(), "con una muestra de", num_test, "municipios")
    print('La predicción es menor que el dato real',abajo, "veces")
    print(test)
    return test

In [ ]:
dfs = obtain_data()
df = create_df(dfs)

print("\nEl area total cubierta por los municipios es: " + str(df.sum(axis = 0, skipna = True)[4]) + " km^2, mientras el area total de la Amazonia es de 5,5M km^2.")
print("Por ello, podemos afirmar que estamos cubriendo una superficie que supone el 92% del total.\n")

plot_map(df, 'mapa.html')

n_vars_temp = df.loc[:,'Deforestacion 2000':].shape[1]//19 # number of variables per year

#shuffle data
df = shuffle_df(df)

results = []
n_replics = [0,1,5,10,50,100]
for n in n_replics:
    if n == 0:
        df_complete = df
    else:
        fake_df = generate_fake_data(df, n)
        df_complete = df.append(fake_df)
    #df_complete = df_complete.sort_values(by=['Municipio']) # to be ommited for the test: 760 primeros municipios son los reales
    df_complete.loc[:,'Latitud':] = df_complete.loc[:,'Latitud':].astype('float')
    df_complete = df_complete.infer_objects()
    
    #plot map
    name = 'mapa_fake' + str(n) + '.html'
    plot_map(df_complete, name)

    #shuffle data
    df_complete[760:] = shuffle_df(df_complete[760:])
    
    # structure data
    fixed_vars = df_complete.loc[:,['Municipio', 'Estado', 'Latitud', 'Longitud', 'Area total', 'Hidrografia', 'No bosque']]
    temporal_vars = df_complete.loc[:,'Deforestacion 2000':]
    window_size=18
    temporal_train, temporal_test = series_to_supervised(temporal_vars, 0, 1, window_size, n_vars_temp)
    
    # preprocess data for training
    X1, X2, y, scaler, scaler2, scaler3 = preprocess_train(temporal_train, fixed_vars, n_vars_temp)
    
    # create model
    hidden_layers=[1,2,3]
    epochs=[5,10,20,50]
    batch_sizes=[32,64]
    for h in hidden_layers:
        for e in epochs:
            for bs in batch_sizes:
                print("Creando el modelo...")
                print("\tNúmero de épocas:", e)
                print('\tBatch size:', bs)
                print('\tCapas intermedias', h)
                print('\tMuestra para training:', len(df_complete), 'municipios')
                model = create_model(window_size, n_vars_temp, h)
                model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])
                # fit model
                train_size = int(len(X1)*0.8)
                checkpoint_cb = ModelCheckpoint("my_keras_model.h5", save_best_only=True, verbose=1)
                model.fit([X1[0:train_size], X2[0:train_size]], y[0:train_size], epochs=e, batch_size=32, verbose=2, validation_data=([X1[train_size:], X2[train_size:]], y[train_size:]), callbacks=[checkpoint_cb])

                # preprocess data for testing
                X1_test, y_test = preprocess_test(temporal_test, n_vars_temp, scaler, scaler2, scaler3)
                X2_test = X2[0:760]
                municipios_reales = fixed_vars.iloc[0:760,0]

                # evaluate model on new data
                model = load_model("my_keras_model.h5")
                y_pred = model.predict([X1_test, X2_test])    
                test = evaluate_test(municipios_reales, y, y_pred)

                d = {}
                d['Municipios'] = len(df_complete)
                d['Epocas'] = e
                d['Batch size'] = bs
                d['Capas intermedias'] = h
                d['Error absoluto medio'] = test['Error'].mean()
                results.append(d)

# Check if data folder already created. If not, create it
if not os.path.exists('resultados/'):
    os.makedirs('resultados/')
df_results = pd.DataFrame(results)
df_results.to_csv('resultados/precision_configuraciones_modelo.csv') # results of the different configurations


In [ ]:
df_results = pd.DataFrame(results)
df_results.to_csv('resultados/precision_configuraciones_modelo.csv') # results of the different configurations
